# How To Exit Emergency Mode Without Root Password On Linux
> I recently had the pleasure of having my laptop reboot into emergency mode after I hard powered it off. The passwords I tried did not work nor could not remember my root password. Hopefully this guide will be helpful for me in the future or for any poor soul who is locked out of emergency mode.
- comments: true

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#How-To-Exit-Emergency-Mode-Without-Root-Password-On-Linux" data-toc-modified-id="How-To-Exit-Emergency-Mode-Without-Root-Password-On-Linux-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>How To Exit Emergency Mode Without Root Password On Linux</a></span></li><li><span><a href="#How-To-Change-The-Root-Password,-When-Root-Password-Is-Unknown" data-toc-modified-id="How-To-Change-The-Root-Password,-When-Root-Password-Is-Unknown-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>How To Change The Root Password, When Root Password Is Unknown</a></span><ul class="toc-item"><li><span><a href="#How-To-Change-Kernel-Parameters" data-toc-modified-id="How-To-Change-Kernel-Parameters-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>How To Change Kernel Parameters</a></span></li><li><span><a href="#How-To-Reset-The-Root-Password" data-toc-modified-id="How-To-Reset-The-Root-Password-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>How To Reset The Root Password</a></span></li></ul></li><li><span><a href="#passwd-root" data-toc-modified-id="passwd-root-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>passwd root</a></span></li><li><span><a href="#Fixing-Emergency-Mode-Error" data-toc-modified-id="Fixing-Emergency-Mode-Error-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Fixing Emergency Mode Error</a></span></li><li><span><a href="#Explanation-Behind-All-Steps" data-toc-modified-id="Explanation-Behind-All-Steps-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Explanation Behind All Steps</a></span><ul class="toc-item"><li><span><a href="#Why-We-Change-Kernel-Parameters" data-toc-modified-id="Why-We-Change-Kernel-Parameters-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Why We Change Kernel Parameters</a></span></li><li><span><a href="#Why-We-Unmount-And-Remount-The-Root-Partition-To-Reset-The-Root-Password" data-toc-modified-id="Why-We-Unmount-And-Remount-The-Root-Partition-To-Reset-The-Root-Password-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Why We Unmount And Remount The Root Partition To Reset The Root Password</a></span></li><li><span><a href="#Why-We-Use-systemctl-Instead-of-journalctl" data-toc-modified-id="Why-We-Use-systemctl-Instead-of-journalctl-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Why We Use <code>systemctl</code> Instead of <code>journalctl</code></a></span></li></ul></li></ul></div>

# How To Change The Root Password, When Root Password Is Unknown

## How To Change Kernel Parameters
Instructions from 
[Milosz Galazka](https://blog.sleeplessbeastie.eu/2014/05/01/how-to-access-single-user-mode-without-password/)


> In case the *Recovery mode*  menu entry is not available, you need to perform five simple steps in order to modify kernel parameters list.
> 
>1. Turn on computer.
>2. Access GRUB menu.
>3. Edit existing menu entry (use  `e`  key).
>4. Add  `single`  keyword (alternatively you can use  `-s`  or  `S` ) to the Linux kernel parameters list.
>5. Press  `CTRL-X`  or  `F10`  while still in edit mode to continue boot process.

## How To Reset The Root Password
Instructions from 
[TecMint](https://blog.sleeplessbeastie.eu/2014/05/01/how-to-access-single-user-mode-without-password/)

> Now check the status of root partition by running following command on the single user mode.
>
>```
># mount | grep root
>```
>![Check Root Mount Status](https://www.tecmint.com/wp-content/uploads/2015/03/Check-Root-Mount-Status.png)
>
> You may notice that root partition is reported to be `'ro'` (Read Only). We need to have read-write permission on root partition to change the root password.
> ```
> # mount -o remount,rw /
> ```
> Also cross check, if the root partition is mounted with read-write permission mode.
>
> ```
> # mount | grep root```
> ![Set Permissions on Root Partition](https://www.tecmint.com/wp-content/uploads/2015/03/Set-Read-Write-Permission-on-Root-Partition.png)
> Now you can change the root password by typing the passwd command. But that is not done. We need to relabel SELinux context. If we skip relabeling the whole SELinux context we would be able to login using using password.
>
> ```
> # passwd root
> [Enter New Password]
> [Re-enter New Password]```
>
> ```# touch /.autorelabel```
> ![Reset root Password](https://www.tecmint.com/wp-content/uploads/2015/03/Reset-root-Password.png)
> Reboot and login again to root account and see if everything works ok or not?
>
> ```# exec /sbin/init```


# Fixing Emergency Mode Error
Instructions from
[ Molesbite Hacks](https://www.youtube.com/watch?v=u42IpZU5ipQ)

> Type `systemctl`
>
> You'll get a list of logs, scroll till you find the error highlighted in red.
>
> Type `fsck /dev/mapper/fedora-root`
>
> Hit Y (yes)

When I initially ran `fsck /dev/mapper/fedora-root`, I got a error that wouldn't allow me to commit the changes

I decided to run `systemctl` and look for the red error

In my case, the error was in `/dev/mapper/fedora-root`

I modified the directory to be `/dev/mapper/fedora-home`, matching the one listed by `systemctl`

# Explanation Behind All Steps

## Why We Change Kernel Parameters
We changed the kernel parameters so that we avoid regular boot, as it would be stuck in emergency boot. 

## Why We Unmount And Remount The Root Partition To Reset The Root Password
As explained in the [TecMint article](https://blog.sleeplessbeastie.eu/2014/05/01/how-to-access-single-user-mode-without-password/), we unmount and remount (with `mount -o remount,rw /`) the root partition so that we have read-write access to modify the root's password.

## Why We Use `systemctl` Instead of `journalctl`
`journalctl` shows logs for everything including boots. If you have multiple boots, you may generate multiple logs. Using `systemctl` focuses on only the system processes, so the list is smaller to scrub through